In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/SemEval_26_Task8_MTRAG
!pwd
!ls

Mounted at /content/drive
/content/drive/MyDrive/SemEval_26_Task8_MTRAG
/content/drive/MyDrive/SemEval_26_Task8_MTRAG
 all_corpus_query_rewrite   history_selected_rewrite_queries   paper_draft.pdf
 beir			    indexes			      'queries data'
 dataset		    indexes_qwen3_emb4b		       README.md
 generator_reference	    logs			       src


INSTALL

In [2]:
!pip -q install -U transformers accelerate sentence-transformers
!pip install -U --no-cache-dir faiss-gpu-cu11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 171.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 212.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 407.5 MB/s eta 0:00:00


CONFIG

In [3]:
import os, re, json
from pathlib import Path
import numpy as np
import faiss
import torch
from functools import lru_cache
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

In [16]:
ROOT = Path("/content/drive/MyDrive/SemEval_26_Task8_MTRAG")
QUERIES_DIR = ROOT / "queries data"
OUT_DIR = ROOT / "all_corpus_query_rewrite"
OUT_DIR.mkdir(parents=True, exist_ok=True)

DATASETS = ["cloud", "fiqa", "govt"]

PRON_MODEL    = "Qwen/Qwen2.5-7B-Instruct-1M"
REWRITE_MODEL = "Qwen/Qwen3-30B-A3B-Instruct-2507"
EMBED_MODEL   = "Qwen/Qwen3-Embedding-4B"


# Behavior knobs （跟ollama一样）
PRON_TEMPERATURE = 0.1
PRON_TOP_P       = 0.9
SEED             = 42
RW_TEMPERATURE   = 0.0
MAX_NEW_PRON     = 64
MAX_NEW_REWRITE  = 128

SELECT_K         = 3
KEEP_RECENT      = 1

In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

torch.manual_seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)

DEVICE: cuda


MODEL

In [8]:
tok_rw = AutoTokenizer.from_pretrained(REWRITE_MODEL, use_fast=True)
model_rw = AutoModelForCausalLM.from_pretrained(
    REWRITE_MODEL,
    dtype=torch.bfloat16,
    max_memory={0: "38GiB", "cpu": "64GiB"},
    device_map="auto",
)

tok_pr = AutoTokenizer.from_pretrained(PRON_MODEL, use_fast=True)
model_pr = AutoModelForCausalLM.from_pretrained(
    PRON_MODEL,
    device_map="auto",
    dtype=torch.bfloat16,
)

embedder = SentenceTransformer(EMBED_MODEL, device="cpu")

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model-00004-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00005-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00007-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

REGEX & PARSING

In [9]:
USER_LINE_RE = re.compile(r"^\|user\|\s*:\s*(.*)\s*$", re.M)
PRON_RE = re.compile(r"\b(he|she|it|they|them|his|her|their|this|that|these|those)\b", re.I)

def build_turn_dict(jsonl_path: Path):
    """
    (cid, turn) -> {"history":[...], "current":"..."}
    """
    d = {}
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            _id = obj["_id"]
            cid, turn = _id.split("<::>")
            turn = int(turn)

            qs = USER_LINE_RE.findall(obj.get("text", ""))
            if not qs:
                continue
            d[(cid, turn)] = {"history": qs[:-1], "current": qs[-1]}
    return d

@torch.inference_mode()
def chat_generate(tokenizer, model, messages, max_new_tokens=128, temperature=0.0, top_p=1.0, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    do_sample = (temperature > 0)
    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id,
    )
    if do_sample:
        gen_kwargs["top_p"] = top_p

    out = model.generate(**inputs, **gen_kwargs)
    out_ids = out[0][len(inputs.input_ids[0]):]
    return tokenizer.decode(out_ids, skip_special_tokens=True).strip()

def extract_json_question(s: str) -> str:
    m = re.search(r"\{.*\}", s, flags=re.S)
    if not m:
        return ""
    try:
        obj = json.loads(m.group(0))
        return (obj.get("question") or "").strip()
    except Exception:
        return ""

def load_done_ids(out_path: Path) -> set[str]:
    """
    Read existing output jsonl and collect already written _id
    """
    done = set()
    if not out_path.exists():
        return done
    with open(out_path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                obj = json.loads(line)
                done.add(str(obj["_id"]))
            except Exception:
                continue
    return done


PRONOUN REWRITE

In [10]:
def rewrite_pronouns(sentence: str, context_hint: str | None = None) -> str:
    if not PRON_RE.search(sentence):
        return sentence

    user = f"""Replace anaphoric pronouns with their most likely explicit referents inferred from the sentence itself{(" and the hint: " + context_hint) if context_hint else ""}.

Rules:
- Preserve facts and meaning.
- Only replace anaphoric pronouns (he, she, it, they, this, that, those, these, his, her, their, etc.) when unambiguous.
- Do NOT add new facts.
- Keep ONE sentence.
- Output ONLY the rewritten sentence.

Sentence: {sentence}
Rewritten:"""

    msgs = [
        {"role":"system","content":"You rewrite text. Follow user rules exactly."},
        {"role":"user","content":user},
    ]
    return chat_generate(
        tok_pr, model_pr, msgs,
        max_new_tokens=MAX_NEW_PRON,
        temperature=PRON_TEMPERATURE,
        top_p=PRON_TOP_P,
        seed=SEED
    )

@lru_cache(maxsize=4096)
def rewrite_if_pronoun(sentence: str, hint: str | None) -> str:
    return rewrite_pronouns(sentence, hint)

Embedding + similarity for select_history

In [14]:
def get_embed(text: str) -> np.ndarray:
    v = embedder.encode([text], prompt_name="query", normalize_embeddings=True)
    return np.asarray(v[0], dtype="float32")

def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.dot(a, b))

def select_history(history, current, k=3, keep_recent=1):
    history_clean, prev = [], None
    for h in history:
        h2 = rewrite_if_pronoun(h, prev)
        history_clean.append(h2)
        prev = h2

    current_clean = rewrite_if_pronoun(current, history_clean[-1] if history_clean else None)

    if not history_clean:
        return [], current_clean
    if len(history_clean) <= k:
        return history_clean, current_clean

    recent = history_clean[-keep_recent:] if keep_recent > 0 else []
    pool = history_clean[:-keep_recent] if keep_recent > 0 else history_clean

    # ---- batch embed (CPU) ----
    cur_vec = embedder.encode([current_clean], prompt_name="query", normalize_embeddings=True)[0]
    pool_vecs = embedder.encode(pool, prompt_name="query", normalize_embeddings=True)

    scores = pool_vecs @ cur_vec
    order = np.argsort(-scores)

    need = max(0, k - len(recent))
    picked = [pool[i] for i in order[:need]]
    return picked + recent, current_clean


(MAIN PART) QUERY REWRITE

In [12]:
def rewrite_standalone(history: list[str], current: str) -> str:
    history_text = "\n".join(f"- {h}" for h in history if h)

    sys = (
        "You are a query rewriter for retrieval. "
        "Output must be a single-line JSON object only, with one key: question. "
        "Never invent facts; use ONLY info from HISTORY and CURRENT."
    )
    user = f"""Rewrite CURRENT into one standalone question.

                Rules:
                - USE ONLY info from HISTORY/CURRENT.
                - Keep names explicit; avoid pronouns where possible.
                - Be minimal; no new terms.
                
                IMPORTANT:
                - If CURRENT is already standalone AND sufficiently specific for document retrieval,
                  return it unchanged.
                - If CURRENT is short, generic, or a yes/no follow-up
                  (e.g., "Do I need scripts?", "Is it required?", "How does it help?"),
                  rewrite it to explicitly include:
                    - the relevant product/system
                    - the relevant task or operation
                  using ONLY information inferable from HISTORY.

HISTORY:
{history_text}

CURRENT:
{current}

Return JSON exactly like:
{{"question": "<one-line question>"}}"""

    msgs = [{"role":"system","content":sys},{"role":"user","content":user}]
    raw = chat_generate(tok_rw, model_rw, msgs, max_new_tokens=MAX_NEW_REWRITE, temperature=RW_TEMPERATURE)
    return extract_json_question(raw)

In [13]:
history_test = [
    "where is bone marrow found what does it do for the body",
    "What happens if it does not work well?",
    "How is Sickle cell treated?",
    "Will it kill me?",
    "How about transplant?",
    "Is Huntington's disease also inherited?"
]
current_test = "Any cures for it?"

sel, cur = select_history(history_test, current_test, k=3, keep_recent=1)
print("\n[TEST] selected history:")
for s in sel: print("-", s)
print("[TEST] current_clean:", cur)
print("[TEST] final rewrite:", rewrite_standalone(sel, cur))


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



[TEST] selected history:
- How is Sickle cell treated?
- Will sickle cell kill me?
- Is Huntington's disease also inherited?
[TEST] current_clean: Any cures for Huntington's disease?
[TEST] final rewrite: Are there any cures for Huntington's disease?


RUN OVER 3 DATASETS

In [ ]:
for ds in DATASETS:
    in_path  = QUERIES_DIR / f"{ds}_questions.jsonl"
    out_path = OUT_DIR / f"{ds}_rewritten_last_turn.jsonl"

    print(f"\n=== Dataset: {ds} ===")
    print("in :", in_path)
    print("out:", out_path)

    data = build_turn_dict(in_path)
    done_ids = load_done_ids(out_path)

    print("turns loaded:", len(data))
    print("already done:", len(done_ids))

    written = 0
    skipped = 0

    with open(out_path, "a", encoding="utf-8") as f:
        for (cid, turn), item in data.items():
            qid = f"{cid}<::>{turn}"

            if qid in done_ids:
                skipped += 1
                continue

            history = item["history"]
            current = item["current"]

            clean_hist = []
            prev = None
            for h in history:
                h2 = rewrite_pronouns(h, prev)
                clean_hist.append(h2)
                prev = h2

            current2 = rewrite_pronouns(current, clean_hist[-1] if clean_hist else None)

            rewritten = rewrite_standalone(clean_hist, current2)
            if not rewritten:
                continue

            rec = {"_id": qid, "text": f"|user|: {rewritten}"}
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
            written += 1

            if written % 50 == 0:
                f.flush()

    print("skipped:", skipped, "new written:", written, "->", out_path)

    # 释放显存
    import gc, torch
    gc.collect()
    torch.cuda.empty_cache()

print("\nALL DONE. Outputs in:", OUT_DIR)
